In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import utils.constants as constants

In [3]:
genes_associations_dir = os.path.join(constants.PREPROCESSED_BASED_DIR, 'gene_associations')

output_dir = os.path.join(constants.RESULTS_DIR, 'roc_validation', 'classifier_tables')
os.makedirs(output_dir, exist_ok=True)

# Load all S-PrediXcan gene mappings

In [4]:
with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load fastENLOC results

In [5]:
spredixcan_genes_associations_filename = os.path.join(genes_associations_dir, 'fastenloc', 'fastenloc-torus-genes_rcp.pkl.xz')
display(spredixcan_genes_associations_filename)

spredixcan_genes_associations = pd.read_pickle(spredixcan_genes_associations_filename)

'/mnt/phenomexcan/results/preprocessed_data/gene_associations/fastenloc/fastenloc-torus-genes_rcp.pkl.xz'

In [6]:
spredixcan_genes_associations = spredixcan_genes_associations.rename(index=genes_mapping_1)

In [7]:
spredixcan_genes_associations.shape

(22233, 4083)

In [8]:
spredixcan_genes_associations.head(5)

,L12_NAILDIS-Nail_disorders,J69-Diagnoses_main_ICD10_J69_Pneumonitis_due_to_solids_and_liquids,R07-Diagnoses_main_ICD10_R07_Pain_in_throat_and_chest,M46-Diagnoses_main_ICD10_M46_Other_inflammatory_spondylopathies,20003_2038460150-Treatmentmedication_code_paracetamol,E10-Diagnoses_main_ICD10_E10_Insulindependent_diabetes_mellitus,5134_raw-6mm_strong_meridian_left,E86-Diagnoses_main_ICD10_E86_Volume_depletion,L12_ERYTHEMATOUSOTH-Other_erythematous_conditions,K83-Diagnoses_main_ICD10_K83_Other_diseases_of_biliary_tract,...,20090_394-Type_of_fatoil_used_in_cooking_Unknown_soft_margarine,22617_3512-Job_SOC_coding_Aircraft_pilots_and_flight_engineers,6034-Target_heart_rate_achieved,20003_1140883066-Treatmentmedication_code_insulin_product,22601_41223241-Job_coding_accounts_and_wages_clerkassistantsupervisor_bookkeeper_cost_or_ledger_clerk_audit_assistant_budget_officer_student_loans_officer_paymaster,I82-Diagnoses_main_ICD10_I82_Other_venous_embolism_and_thrombosis,20107_12-Illnesses_of_father_Severe_depression,B07-Diagnoses_main_ICD10_B07_Viral_warts,22601_12253140-Job_coding_sports_centre_manager_riding_school_owner_sports_ground_manager_baths_manager,2664_2-Reason_for_reducing_amount_of_alcohol_drunk_Doctors_advice
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000121410,1.515300e-08,2.216225e-08,0.005937,5.482960e-10,0.006081,0.000458,0.006905,2.241375e-10,1.753100e-09,2.774200e-05,...,0.000292,1.498710e-10,6.837000e-06,0.000714,1.503420e-10,5.317725e-08,5.242997e-04,1.807621e-08,7.545600e-08,0.000154
ENSG00000148584,2.796517e-11,1.696862e-10,0.001970,1.021794e-10,0.010776,0.000006,0.005850,7.833300e-11,2.507511e-10,3.895151e-10,...,0.000225,1.869519e-10,4.081060e-11,0.000270,2.783110e-11,1.083806e-10,3.367239e-07,5.118762e-11,1.050704e-10,0.000449
ENSG00000175899,1.926980e-08,7.858200e-09,0.000947,5.466000e-11,0.003225,0.000025,0.003633,4.745490e-10,5.610300e-10,9.633650e-06,...,0.000081,1.608430e-10,3.141280e-06,0.000401,1.096541e-10,3.252662e-08,5.695800e-04,1.128942e-08,1.132000e-08,0.000130
ENSG00000166535,1.142270e-09,3.492600e-09,0.000428,1.476100e-10,0.004078,0.000247,0.018624,3.055700e-10,3.589500e-10,3.792130e-06,...,0.000197,2.759200e-10,7.272498e-06,0.000148,1.642360e-11,8.912710e-08,8.749040e-04,1.326800e-10,2.235800e-09,0.000005
ENSG00000184389,1.997480e-08,5.515000e-10,0.002141,2.412720e-09,0.004778,0.000200,0.141350,1.314787e-10,1.597080e-09,2.149724e-05,...,0.000311,2.357997e-10,1.826130e-09,0.000989,6.655500e-11,1.471335e-10,6.330287e-04,4.799799e-09,1.115040e-07,0.000050


In [9]:
assert spredixcan_genes_associations.isna().sum().sum() == 0

In [10]:
assert not spredixcan_genes_associations.isin([np.inf, -np.inf]).any().any()

In [11]:
_all_values = pd.Series(spredixcan_genes_associations.values.flatten())

In [12]:
_tmp = _all_values.sort_values(ascending=True)

In [13]:
display(_tmp.head())

63673277    0.0
13822236    0.0
13822237    0.0
13822238    0.0
13822239    0.0
dtype: float64

In [14]:
_tmp = _all_values.sort_values(ascending=False)

In [15]:
display(_tmp.head())

84487107    2.661831
29863101    2.531200
41614643    2.484566
53648952    2.129925
53649340    2.116626
dtype: float64

# Load OMIM silver standard

In [16]:
omim_silver_standard = pd.read_csv(os.path.join(constants.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [17]:
omim_silver_standard = omim_silver_standard.dropna(subset=['ensembl_gene_id', 'trait', 'pheno_mim'])

In [18]:
display(omim_silver_standard.shape)
display(omim_silver_standard.head())

(7809, 7)

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


### Save list of mapped UK Biobank traits

In [19]:
from clustering.biclustering.analysis import Trait

In [20]:
ukb_traits_mapped = pd.Series(omim_silver_standard['trait'].unique())
display(ukb_traits_mapped.head())

0               M41-Diagnoses_main_ICD10_M41_Scoliosis
1            H80-Diagnoses_main_ICD10_H80_Otosclerosis
2    20002_1226-Noncancer_illness_code_selfreported...
3    I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic...
4    20002_1136-Noncancer_illness_code_selfreported...
dtype: object

In [21]:
ukb_traits_mapped.shape

(107,)

In [22]:
ukb_codes_mapped = [Trait(t).trait_code for t in ukb_traits_mapped]

In [23]:
len(ukb_codes_mapped)

107

In [24]:
ukb_codes_mapped[:5]

['M41', 'H80', '20002_1226', 'I25', '20002_1136']

In [25]:
# This is needed to run gwas2gene (Yanyu's scripts)
pd.Series(ukb_codes_mapped).to_csv('/mnt/tmp/selected_ukb_traits_omim.txt', index=False, header=False)

# Read gwas2gene (Yanyu) results

In [26]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [27]:
gwas2gene_results_dir = '/mnt/phenomexcan/results/roc_validation/ukb_gwas2gene_results_omim_silver_standard/'

In [28]:
readRDS = robjects.r['readRDS']

In [29]:
f_files = glob(os.path.join(gwas2gene_results_dir, '*.rds'))
display(len(f_files))

if len(f_files) != len(ukb_codes_mapped):
    print(f'WARNING: some files are not there. {len(ukb_codes_mapped)} expected, {len(f_files)} found.')

99

In [30]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code = f_base.split('.')[0]
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = spredixcan_genes_associations.index.intersection(set(f_gene_list))

In [31]:
len(gwas2genes_results)

99

In [32]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

# gwas2gene_all_genes = spredixcan_genes_associations.index.intersection(gwas2gene_all_genes)
# display(len(gwas2gene_all_genes))

19179

9503

# Create list of UKB-OMIM traits

In [33]:
omim_silver_standard.head()

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


# Create PrediXcan classifier table

In [34]:
_tmp = omim_silver_standard[['trait', 'ensembl_gene_id']]
ukb_traits_common = _tmp['trait'].unique()

omim_true_classes = _tmp[['trait', 'ensembl_gene_id']].drop_duplicates()
omim_true_classes = omim_true_classes.assign(omim_value=1)
omim_true_classes = omim_true_classes.set_index(['trait', 'ensembl_gene_id'])

In [35]:
len(ukb_traits_common)

107

In [36]:
omim_true_classes.shape

(7046, 1)

In [37]:
omim_true_classes.head()

omim_value
trait                                  ensembl_gene_id            
M41-Diagnoses_main_ICD10_M41_Scoliosis ENSG00000108946           1
                                       ENSG00000134250           1
                                       ENSG00000114062           1
                                       ENSG00000198467           1
                                       ENSG00000154864           1

In [38]:
len(ukb_traits_common)

107

In [39]:
index_tuples = []

for t in ukb_traits_common:
    t_code = Trait(t).trait_code
    if t_code not in gwas2genes_results:
        continue
    
    for g in gwas2genes_results[t_code]:
        index_tuples.append((t, g))

In [40]:
len(index_tuples)

19179

In [41]:
index_tuples[:5]

[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000173208'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075089'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000133597'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000111863'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000174606')]

In [42]:
classifier_index = pd.MultiIndex.from_tuples(
    index_tuples,
    names=['ukb_efo', 'gene']
)

In [43]:
len(gwas2gene_all_genes)

9503

In [44]:
classifier_index.shape

(19179,)

In [45]:
predixcan_classifier_df = pd.DataFrame(index=classifier_index, columns=['score', 'predicted_class', 'true_class'])

In [46]:
predixcan_classifier_df = predixcan_classifier_df.sort_index()

In [47]:
predixcan_classifier_df.shape

(19179, 3)

In [48]:
predixcan_classifier_df['true_class'] = 0

In [49]:
predixcan_classifier_df.head()

score predicted_class  true_class
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617   NaN             NaN           0
                            ENSG00000003756   NaN             NaN           0
                            ENSG00000004534   NaN             NaN           0
                            ENSG00000004838   NaN             NaN           0
                            ENSG00000004897   NaN             NaN           0

In [50]:
true_classes = omim_true_classes.squeeze()
display(true_classes.shape)
display(true_classes.head())

(7046,)

trait                                   ensembl_gene_id
M41-Diagnoses_main_ICD10_M41_Scoliosis  ENSG00000108946    1
                                        ENSG00000134250    1
                                        ENSG00000114062    1
                                        ENSG00000198467    1
                                        ENSG00000154864    1
Name: omim_value, dtype: int64

In [51]:
predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(true_classes.index), 'true_class'] = 1

In [52]:
assert predixcan_classifier_df['true_class'].isna().sum() == 0

In [53]:
# some testing
predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis',)].head()

,score,predicted_class,true_class
gene,,,
ENSG00000012504,NaN,NaN,0
ENSG00000075089,NaN,NaN,0
ENSG00000075336,NaN,NaN,0
ENSG00000078401,NaN,NaN,0
ENSG00000090263,NaN,NaN,0


In [54]:
true_classes.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234')]

1

In [55]:
'ENSG00000090263' not in true_classes.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].index

True

In [56]:
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234'), 'true_class'] == 1.0
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263'), 'true_class'] == 0.0

In [57]:
len(gwas2gene_all_genes)

9503

In [58]:
# score
df_score = pd.Series(index=classifier_index)

for trait in ukb_traits_common:
    trait_code = Trait(trait).trait_code
    if trait_code not in gwas2genes_results:
        print(trait_code)
        continue
    trait_genes = gwas2genes_results[trait_code]
    scores = spredixcan_genes_associations.loc[trait_genes, trait]
    df_score.loc[trait] = scores.values

M13
I95
I71
20002_1264
20002_1081
22130
C80
20002_1538


In [59]:
# some testing
df_score = df_score.dropna().sort_index()
assert df_score.isna().sum().sum() == 0

In [60]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    2.563019e-02
                             ENSG00000003756    7.435000e-02
                             ENSG00000004534    4.208160e-01
                             ENSG00000004838    4.417000e-04
                             ENSG00000004897    1.116773e-09
dtype: float64

In [61]:
# some testing
_gene, _trait = ('ENSG00000090263', 'M41-Diagnoses_main_ICD10_M41_Scoliosis')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [62]:
_gene, _trait = ('ENSG00000070061', 'O14-Diagnoses_main_ICD10_O14_Gestational_pregnancyinduced_hypertension_with_significant_proteinuria')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [63]:
df_score.shape

(19179,)

In [64]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    2.563019e-02
                             ENSG00000003756    7.435000e-02
                             ENSG00000004534    4.208160e-01
                             ENSG00000004838    4.417000e-04
                             ENSG00000004897    1.116773e-09
dtype: float64

In [65]:
df_score.min()

0.0

In [66]:
df_score.max()

1.759235608

In [67]:
predixcan_classifier_df = predixcan_classifier_df.assign(score=df_score)

In [68]:
assert not predixcan_classifier_df['score'].isna().any()

In [69]:
from scipy import stats

In [70]:
_n_genes = len(gwas2gene_all_genes)
display(_n_genes)

SCORE_THRESHOLD = 0.1
display(SCORE_THRESHOLD)

9503

0.1

In [71]:
predixcan_classifier_df = predixcan_classifier_df.assign(predicted_class=(predixcan_classifier_df['score'] > SCORE_THRESHOLD).astype(int))

In [72]:
predixcan_classifier_df.shape

(19179, 3)

In [73]:
predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [74]:
predixcan_classifier_df.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].sort_values('true_class', ascending=False).head()

,score,predicted_class,true_class
gene,,,
ENSG00000157764,6.912200e-08,0,1
ENSG00000162769,3.231060e-06,0,1
ENSG00000112234,6.222500e-07,0,1
ENSG00000012504,1.772800e-11,0,0
ENSG00000173208,1.777500e-07,0,0


## Select genes per trait

In [75]:
#selected_predixcan_classifier_df = predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(trait_genes_to_keep)]
selected_predixcan_classifier_df = predixcan_classifier_df

In [76]:
# some testing

In [77]:
selected_predixcan_classifier_df.shape

(19179, 3)

In [78]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [79]:
selected_predixcan_classifier_df.sort_values('predicted_class').head()

score  \
ukb_efo                                            gene                            
1200-Sleeplessness_insomnia                        ENSG00000001617  2.563019e-02   
20002_1461-Noncancer_illness_code_selfreported_... ENSG00000168505  4.465000e-08   
                                                   ENSG00000166197  3.118300e-06   
                                                   ENSG00000166189  2.720410e-06   
                                                   ENSG00000166171  1.307593e-06   

                                                                    predicted_class  \
ukb_efo                                            gene                               
1200-Sleeplessness_insomnia                        ENSG00000001617                0   
20002_1461-Noncancer_illness_code_selfreported_... ENSG00000168505                0   
                                                   ENSG00000166197                0   
                                                   ENSG00000166189                0   
                                                   ENSG00000166171                0   

                                                                    true_class  
ukb_efo                                            gene                         
1200-Sleeplessness_insomnia                        ENSG00000001617           0  
20002_1461-Noncancer_illness_code_selfreported_... ENSG00000168505           0  
                                                   ENSG00000166197           0  
                                                   ENSG00000166189           0  
                                                   ENSG00000166171           0

In [80]:
selected_predixcan_classifier_df.sort_values('predicted_class', ascending=False).head()

score  \
ukb_efo                                            gene                        
K51-Diagnoses_main_ICD10_K51_Ulcerative_colitis    ENSG00000206503  0.789828   
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000151320  0.887210   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000100417  0.145200   
                                                   ENSG00000100418  0.209020   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000077549  0.250544   

                                                                    predicted_class  \
ukb_efo                                            gene                               
K51-Diagnoses_main_ICD10_K51_Ulcerative_colitis    ENSG00000206503                1   
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000151320                1   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000100417                1   
                                                   ENSG00000100418                1   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000077549                1   

                                                                    true_class  
ukb_efo                                            gene                         
K51-Diagnoses_main_ICD10_K51_Ulcerative_colitis    ENSG00000206503           0  
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000151320           0  
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000100417           0  
                                                   ENSG00000100418           0  
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000077549           0

In [81]:
_tmp = selected_predixcan_classifier_df.sort_values(['true_class', 'ukb_efo'], ascending=False)
display(_tmp.shape)
display(_tmp[_tmp['true_class'] > 0].shape)
display(_tmp[_tmp['true_class'] > 0].head())

(19179, 3)

(125, 3)

score  \
ukb_efo                                            gene                            
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305  5.758000e-06   
                                                   ENSG00000140443  4.758100e-06   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  5.454000e-11   
                                                   ENSG00000183287  1.143380e-10   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183  2.062000e-03   

                                                                    predicted_class  \
ukb_efo                                            gene                               
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305                0   
                                                   ENSG00000140443                0   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812                0   
                                                   ENSG00000183287                0   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183                0   

                                                                    true_class  
ukb_efo                                            gene                         
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305           1  
                                                   ENSG00000140443           1  
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812           1  
                                                   ENSG00000183287           1  
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183           1

### Test classes

In [82]:
selected_predixcan_classifier_df.index.get_level_values('ukb_efo').unique().shape

(99,)

In [83]:
selected_predixcan_classifier_df.index.get_level_values('gene').unique().shape

(9503,)

In [84]:
_pheno = 'N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney_and_ureter'

In [85]:
_clinvar_asthma_genes = omim_silver_standard[omim_silver_standard['trait'] == _pheno]['ensembl_gene_id'].unique()
display(_clinvar_asthma_genes)
display(_clinvar_asthma_genes.shape)

array(['ENSG00000075891', 'ENSG00000130600', 'ENSG00000269821',
       'ENSG00000129757', 'ENSG00000143473', 'ENSG00000275410',
       'ENSG00000019186', 'ENSG00000134371', 'ENSG00000036828',
       'ENSG00000088256', 'ENSG00000187091', 'ENSG00000122194',
       'ENSG00000138592', 'ENSG00000040531', 'ENSG00000138079',
       'ENSG00000021488', 'ENSG00000090402', 'ENSG00000131482',
       'ENSG00000137700', 'ENSG00000164007', 'ENSG00000113946',
       'ENSG00000116039', 'ENSG00000168000', 'ENSG00000123191',
       'ENSG00000171365', 'ENSG00000165704', 'ENSG00000042753',
       'ENSG00000075643', 'ENSG00000169692', 'ENSG00000109667',
       'ENSG00000131183', 'ENSG00000109062', 'ENSG00000149257',
       'ENSG00000198931', 'ENSG00000157388', 'ENSG00000167207',
       'ENSG00000124827', 'ENSG00000134873'], dtype=object)

(38,)

In [86]:
_tmp = selected_predixcan_classifier_df.loc[_pheno]
_tmp.loc[_tmp.index.intersection(_clinvar_asthma_genes)]

,score,predicted_class,true_class
ENSG00000131183,0.002062,0,1


In [87]:
_predixcan_asthma_genes = selected_predixcan_classifier_df.loc[_pheno]

In [88]:
_predixcan_asthma_genes.head()

,score,predicted_class,true_class
gene,,,
ENSG00000005187,2.666423e-09,0,0
ENSG00000005189,3.437330e-09,0,0
ENSG00000011638,3.278863e-09,0,0
ENSG00000027847,2.658019e-08,0,0
ENSG00000048140,1.244200e-08,0,0


In [89]:
selected_predixcan_classifier_df.shape

(19179, 3)

In [90]:
selected_predixcan_classifier_df['predicted_class'].value_counts()

0    18150
1     1029
Name: predicted_class, dtype: int64

In [91]:
selected_predixcan_classifier_df['true_class'].value_counts()

0    19054
1      125
Name: true_class, dtype: int64

In [92]:
selected_predixcan_classifier_df.sort_values(['true_class'], ascending=[False])

score  \
ukb_efo                                            gene                            
20002_1610-Noncancer_illness_code_selfreported_... ENSG00000091137  1.268000e-10   
M06-Diagnoses_main_ICD10_M06_Other_rheumatoid_a... ENSG00000204498  1.959000e-04   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000144061  3.351189e-04   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000171316  2.779820e-03   
M41-Diagnoses_main_ICD10_M41_Scoliosis             ENSG00000162769  3.231060e-06   
                                                   ENSG00000157764  6.912200e-08   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000171862  2.590036e-03   
M41-Diagnoses_main_ICD10_M41_Scoliosis             ENSG00000112234  6.222500e-07   
20002_1197-Noncancer_illness_code_selfreported_... ENSG00000131183  6.507000e-03   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000184937  6.341001e-01   
20002_1197-Noncancer_illness_code_selfreported_... ENSG00000019186  2.805447e-04   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000141646  9.968244e-07   
1980-Worrier_anxious_feelings                      ENSG00000120071  3.253025e-02   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000187266  8.063000e-06   
20002_1162-Noncancer_illness_code_selfreported_... ENSG00000143921  5.035000e-02   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000042832  2.323000e-01   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000138823  1.513786e-04   
M06-Diagnoses_main_ICD10_M06_Other_rheumatoid_a... ENSG00000134242  1.814508e-04   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000196557  1.938982e-01   
20002_1466-Noncancer_illness_code_selfreported_... ENSG00000124564  8.169001e-03   
20002_1162-Noncancer_illness_code_selfreported_... ENSG00000104936  4.329474e-01   
M05-Diagnoses_main_ICD10_M05_Seropositive_rheum... ENSG00000204498  9.828148e-04   
1980-Worrier_anxious_feelings                      ENSG00000167371  2.583516e-05   
20002_1162-Noncancer_illness_code_selfreported_... ENSG00000005471  5.231356e-01   
1980-Worrier_anxious_feelings                      ENSG00000179344  2.197727e-05   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000204370  4.424840e-08   
                                                   ENSG00000132170  4.760181e-05   
L40-Diagnoses_main_ICD10_L40_Psoriasis             ENSG00000204525  1.528300e-10   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000168610  4.683271e-06   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000148795  4.261000e-04   
...                                                                          ...   
20002_1075-Noncancer_illness_code_selfreported_... ENSG00000076513  6.152350e-04   
                                                   ENSG00000073711  4.444946e-04   
                                                   ENSG00000073008  5.314884e-05   
                                                   ENSG00000069399  4.323610e-05   
                                                   ENSG00000065135  6.566419e-05   
                                                   ENSG00000062370  4.800711e-05   
                                                   ENSG00000041515  7.859260e-03   
                                                   ENSG00000031698  3.676001e-02   
                                                   ENSG00000026652  5.199680e-24   
                                                   ENSG00000023445  5.608100e-05   
                                                   ENSG00000012061  1.218787e-05   
                                                   ENSG00000085491  9.430614e-07   
                                                   ENSG00000085511  4.221274e-23   
                                                   ENSG00000089009  4.905700e-05   
                                                   ENSG00000104853  3.614558e-05   
                                           

# Save classifier table

In [93]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [94]:
selected_predixcan_classifier_df.shape

(19179, 3)

In [95]:
selected_predixcan_classifier_df.to_csv(
    os.path.join(output_dir, 'fastenloc-torus-classifier_data.tsv.gz'),
    sep='\t', index=False
)

# ROC curve

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
accuracy_score(selected_predixcan_classifier_df['true_class'], selected_predixcan_classifier_df['predicted_class'])

In [ ]:
roc_auc = roc_auc_score(selected_predixcan_classifier_df['true_class'], selected_predixcan_classifier_df['score'])
display(roc_auc)

In [ ]:
fpr, tpr, thresholds = roc_curve(selected_predixcan_classifier_df['true_class'], selected_predixcan_classifier_df['score'])

In [ ]:
len(fpr)

In [ ]:
with sns.plotting_context('poster'):
    plt.figure(figsize=(10, 10))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('fastENLOC performance using OMIM')
    plt.legend(loc="lower right")

In [ ]:
pd.DataFrame({
    'method': 'MultiXcan',
    'tpr': tpr,
    'fpr': fpr,
}).to_csv('/mnt/tmp/roc_data_fastenloc.tsv.gz', sep='\t', index=False)

# Precision-recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import minmax_scale

In [ ]:
score_min = selected_predixcan_classifier_df['score'].min()
score_max = selected_predixcan_classifier_df['score'].max()
score_len = selected_predixcan_classifier_df['score'].shape[0]

random_score = minmax_scale(np.random.rand(int(score_len * 100)), feature_range=(score_min, score_max))
random_prediction = (random_score > (random_score.mean())).astype(int)
display(random_score.shape)
#assert selected_predixcan_classifier_df.shape[0] == random_score.shape[0]

In [ ]:
_random_score_single = minmax_scale(np.random.rand(int(score_len)), feature_range=(score_min, score_max))
ap = average_precision_score(selected_predixcan_classifier_df['true_class'], _random_score_single)
display(f'Random average precision score: {ap:.3f}')

In [ ]:
ap = average_precision_score(selected_predixcan_classifier_df['true_class'], selected_predixcan_classifier_df['score'])
display(f'fastENLOC average precision score: {ap:.3f}')

In [ ]:
precision, recall, thresholds = precision_recall_curve(selected_predixcan_classifier_df['true_class'], selected_predixcan_classifier_df['score'])

In [ ]:
len(precision)

In [ ]:
random_precision = []
random_recall = []

for i in range(10):
    random_score = minmax_scale(np.random.rand(score_len), feature_range=(score_min, score_max))
    rp, rr, _ = precision_recall_curve(selected_predixcan_classifier_df['true_class'], random_score)
    random_precision.extend(rp)
    random_recall.extend(rr)

random_precision = np.array(random_precision)
random_recall = np.array(random_recall)

In [ ]:
random_recall.shape

In [ ]:
df_plot_multixcan = pd.DataFrame({
    'Method': 'fastENLOC',
    'Precision': precision,
    'Recall': recall,
})

df_plot_random = df_plot = pd.DataFrame({
    'Method': 'Random',
    'Precision': random_precision,
    'Recall': random_recall,
})

df_plot = pd.concat((df_plot_multixcan, df_plot_random), ignore_index=True)
#assert df_plot.shape == (df_plot_multixcan.shape[0] + df_plot_random.shape[0], 3)
df_plot.head()

In [ ]:
df_plot[df_plot['Method'] == 'fastENLOC'].sort_values('Recall').head()

In [ ]:
df_plot['Method'].value_counts()

In [ ]:
with sns.plotting_context('talk'):
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.lineplot(x='Recall', y='Precision', label='fastENLOC', data=df_plot[df_plot['Method'] == 'fastENLOC'], ax=ax)
    sns.lineplot(x='Recall', y='Precision', label='Random', data=df_plot[df_plot['Method'] == 'Random'], ax=ax)
    ax.set_ylim(0.0, 0.15)